## Exploring experiments

In [1]:
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from pymongo import MongoClient

/home/Edu/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/Edu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# config logging
logging.basicConfig(level=logging.INFO)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [3]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [9]:
# get mnist experiments with high success rate
# run detector
# compute performance
mnist = con.find({'dataset.name': 'MNIST',
                  'metrics.patch_success_rate': {"$gt": 0.90}})

In [10]:
# get mnist experiments with low success rate
# run detector
# compute performance
baselines = con.find({'dataset.name': 'MNIST',
                      'metrics.patch_success_rate': {"$exists": False}})

In [11]:
for b in baselines:
    directory = b['directory']
    print(b['metrics']['accuracy_score'], directory)
    
    model, dataset, metadata = experiment.load(directory)
    clean = dataset.load_clean()

    try:
        (sms_model, outs, recovered, sample, test, flips,
            mode_changes) = saliency.detect(model, clean, random_trials=100)
    except ValueError as e:
        print('error', e)
    else:
        print(flips, mode_changes)

0.9861 27-Apr-2018@03-13-57


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.015
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.0, 0.0) 0.0
0.9857 27-Apr-2018@03-30-26


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.027
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.08899999999999997, 0.031288975694324025) 0.1
0.9858 27-Apr-2018@03-31-31


...

error Input contains NaN, infinity or a value too large for dtype('float32').
0.9866 27-Apr-2018@04-56-56


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.027
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 9 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.006000000000000001, 0.023748684174075836) 0.0
0.9864 27-Apr-2018@04-58-58


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.024
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.0, 0.0) 0.0


In [ ]:
for m in mnist:
    directory = m['directory']
    print(m['metrics']['patch_success_rate'], directory)
    
    model, dataset, metadata = experiment.load(directory)
    clean = dataset.load_clean()

    (sms_model, outs, recovered, sample, test, flips,
            mode_changes) = saliency.detect(model, clean, random_trials=100)
    
    print(flips, mode_changes)

0.9806763285024155 27-Apr-2018@03-02-59


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.018
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.555, 0.10805091392487154) 0.6
0.9851778656126482 27-Apr-2018@03-15-12


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.024
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.8350000000000003, 0.07664854858377945) 0.9
0.9997804128238911 27-Apr-2018@03-30-51


...

INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.019
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 9 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.552, 0.11443775600735975) 0.6
0.9995608256477821 27-Apr-2018@04-43-29


INFO:trojan_defender.detect.saliency_:Computing saliency...
INFO:trojan_defender.detect.saliency_:Finding outleirs...
INFO:trojan_defender.detect.saliency_:Recovering mask...
INFO:trojan_defender.detect.saliency_:Mask proportion is 0.101
INFO:trojan_defender.detect.saliency_:Sampling one observation per class in the clean dataset...
INFO:trojan_defender.detect.saliency_:Predictions are: [0 1 2 3 4 5 6 7 8 9]
INFO:trojan_defender.detect.saliency_:Running trials...


(0.14299999999999996, 0.08277076778670113) 0.1
0.98989898989899 27-Apr-2018@04-57-22


INFO:trojan_defender.detect.saliency_:Computing saliency...
